### Predict House Prices with positron

In [ ]:
import sys
sys.path.insert(1, '../positron/')
import numpy as np
from sklearn.datasets import load_boston


data = load_boston()
# print(data["DESCR"])
X = np.array(data.data)
y = np.array([data.target]).T
print("Loading in the data")
print("X =", X.shape, "y =", y.shape)

Prepare the data. Normalize y values to be between [0;1].

(Later the networks outputs must be scaled up from that)

In [ ]:
import prep


y_orig = np.copy(y)
y = prep.normalize(y)
print("Example outputs (non-normalized):", y_orig[0])
print("Example outputs (normalized):", y[0])

X_train, X_test, y_train, y_test = prep.split_train_test(X, y, 0.1)
print("X_train =", X_train.shape)
print("X_test =", X_test.shape)
print("y_train =", y_train.shape)
print("y_test =", y_test.shape)

Create the network

In [ ]:
import deep


weight_sizes = [10, 50, 100, 50, 10, 1]
ws, bs = deep.init_network(X.shape, weight_sizes)

for w, b in zip(ws, bs):
    print(w.shape, b.shape)

In [ ]:
import activ


activfs = [activ.sigmoid, activ.sigmoid, activ.sigmoid]
dactivfs = [activ.dsigmoid, activ.dsigmoid, activ.dsigmoid]
print(activfs)

In [ ]:
import score


cost = score.mse
dcost = score.dmse
print(cost)

In [ ]:
import visual


deep.seed(1)
ws, bs = deep.init_network(X_train.shape, weight_sizes)
ws, bs, cost_history = deep.SGD(
    X_train,
    y_train,
    ws,
    bs,
    activfs,
    dactivfs,
    cost,
    dcost,
    epochs=10,
    eta=0.01,
    mini_batch_size=len(X_train),
    verbose=True,
    cost_history_needed=True)

# visual.show_plot(cost_history)

o = deep.feedforward(X_test, ws, bs, activfs)
err = np.abs(o - y_test)
print(f"Max error = {np.max(err)}")
print(f"Min error = {np.min(err)}")
o = o * np.max(y_orig)
print("Examples:")
for i in range(5):
    print("\tOutput =", np.round(o[i]), "Desired Output", y_orig[i])